In [102]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [103]:
load_dotenv(override=True)

True

In [104]:
api_key = os.getenv("OPENAI_API_KEY")
files = [os.getenv("SOURCE_PDF")]
pages = []
store = os.getenv("SOURCE_STORE")

# Files

In [105]:
for file in files:
    loader = PyPDFLoader(file)
    pages.extend(loader.load())

# Recursive TextSplit

In [106]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [107]:
documents = recur_split.split_documents(pages)

# Embbiding

In [108]:
embeddings_model = OpenAIEmbeddings()

# Vector Store

In [109]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=store
)

# LLM

In [110]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

# Chain

In [111]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

# Question

In [112]:
question = "O que é Identificar oportunidades?"

# Answer

In [113]:
chat_chain.invoke({"query": question})

{'query': 'O que é Identificar oportunidades?',
 'result': 'Identificar oportunidades é usar a sua imaginação e habilidades para visualizar oportunidades de criação de valor, como a identificação de necessidades e desafios que precisam ser atendidos.'}

# Prompt

In [134]:
chain_prompt = PromptTemplate.from_template(
"""
Você é um assistente educacional.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
- A disciplina;
- O conteúdo;
- O objetivo da aula.

Com base no documento ENTRECOMP (que está no contexto abaixo), identifique **três competências empreendedoras** que se relacionam diretamente com essa aula.
Retorne apenas os **nomes das competências**, separados por vírgula e a última separada por e.
Exemplo de formato: Visão, Pensamento ético e sustentável e Valorizar ideias.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

# Chain

In [135]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":chain_prompt},
    return_source_documents=True
)

# Question

In [140]:
question = "Aula de <Educação Religiosa>, sendo o conteúdo <Nomes> e o objetivo da aula é <Importância dos Nomes>."

# Answer

In [141]:
chat_chain.invoke({"query": question})

{'query': 'Aula de <Educação Religiosa>, sendo o conteúdo <Nomes> e o objetivo da aula é <Importância dos Nomes>.',
 'result': 'Visão, Valorizar ideias e Consciência de si e autoeficácia.',
 'source_documents': [Document(metadata={'page': 0, 'page_label': '1', 'moddate': '2025-04-28T16:59:30-03:00', 'source': 'D:\\Visual Code\\RAG\\pdf\\entrecomp.pdf', 'doc_id': 1, 'author': 'Roselaine Monteiro Moraes', 'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'total_pages': 2, 'creationdate': '2025-04-28T16:59:30-03:00'}, page_content='significativos. \n \nVisão: é trabalhar para uma visão de futuro, imaginar o futuro, desenvolver um olhar \npara transformar ideias em ações e visualizar cenários futuros para ajudar a orientar \nesforços e iniciativas. \n \nValorizar ideias: é fazer o máximo com as ideias e as oportunidades, avaliar que \nvalor existe em termos sociais, culturais e econômicos, reconhecer o potencial de \ncriação de valor de uma 